## Objective of this Notebook
<br>
Trying to find Better Initial Centroids for K-Means Clustering so that the number of iterations required to converge on the final Centroids are lesser hence the process is faster

### Reading the Data
Information about the Dataset: https://archive.ics.uci.edu/ml/datasets/3D+Road+Network+(North+Jutland,+Denmark)

In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('3D_spatial_network.csv', header=None)

In [11]:
data.columns = ['OSM_ID', 'LONGITUDE', 'LATTITUDE', 'ALTITUDE']

In [12]:
data.head()

,OSM_ID,LONGITUDE,LATTITUDE,ALTITUDE
0,144552912,9.349849,56.740876,17.052772
1,144552912,9.350188,56.740679,17.614840
2,144552912,9.350549,56.740544,18.083536
3,144552912,9.350806,56.740484,18.279465
4,144552912,9.351053,56.740486,18.422974


### Exploratory Data Analysis

In [7]:
import libraryEDA as lib

In [8]:
lib.EDA(data)

Number of Rows in the DataFrame: 434874

Number of Columns in the DataFrame: 4

Descriptive Statistics of the DataFrame are as follows:
                  0              1              2              3
count  4.348740e+05  434874.000000  434874.000000  434874.000000
mean   9.786998e+07       9.731836      57.083758      22.185405
std    3.775233e+07       0.627349       0.289479      18.617989
min    4.482444e+06       8.146126      56.582486      -8.608184
25%    8.267897e+07       9.337649      56.846049       7.028053
50%    1.019797e+08       9.887195      57.042498      17.574678
75%    1.259547e+08      10.172359      57.308669      31.810224
max    1.577424e+08      11.199326      57.750511     134.441947

Columns with Missing Values
Empty DataFrame
Columns: [Column Name, # Missing Values]
Index: []

DataTypes of all columns
0      int64
1    float64
2    float64
3    float64
dtype: object


No missing values

### Data Preperation

There are no missing values or classes in the data.<br>
We normalize the data and drop the column OSM_ID as that is not useful for our clustering

In [15]:
data.drop(labels='OSM_ID', axis=1, inplace=True)

In [17]:
from sklearn import preprocessing

In [18]:
sc = preprocessing.MinMaxScaler()

In [19]:
data_sc = sc.fit_transform(data.values)

In [20]:
data = pd.DataFrame(data_sc)

In [21]:
data.head()

,0,1,2
0,0.394249,0.135605,0.179384
1,0.394361,0.135436,0.183314
2,0.394479,0.135321,0.186590
3,0.394563,0.135270,0.187960
4,0.394644,0.135272,0.188963


### Process of finding better initial centroids for K-Means Clustering

Step 1: Pick samples from the dataset<br>
Step 2: Run K-Means algorithm iterations n number of times on it<br>

In [22]:
SAMPLE_SIZE = 0.1
RANDOM_STATE = 42
NUM_CLUSTERS = 10     # k
NUM_ITER = 3          # n
NUM_ATTEMPTS = 5      # m

data_sample = data.sample(frac=SAMPLE_SIZE, random_state=RANDOM_STATE, replace=False)
data_sample.shape

(43487, 3)

In [23]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=NUM_CLUSTERS, init='random', max_iter=1, n_init=1)#, verbose=1)
km.fit(data_sample)

print('Pre-clustering metrics')
print('----------------------')
print('Inertia:', km.inertia_)
print('Centroids:', km.cluster_centers_)

Pre-clustering metrics
----------------------
Inertia: 1012.60352115
Centroids: [[ 0.33255532  0.38687508  0.13979188]
 [ 0.17580137  0.07500607  0.2294287 ]
 [ 0.15458061  0.25273311  0.18798365]
 [ 0.64840057  0.55808029  0.13148556]
 [ 0.45406389  0.08958653  0.26582884]
 [ 0.5699669   0.3745683   0.13456748]
 [ 0.69634176  0.29119515  0.09901808]
 [ 0.56485591  0.26679838  0.39509002]
 [ 0.67075957  0.81857143  0.24085752]
 [ 0.68648569  0.66983186  0.24874021]]


In [24]:
final_cents = []
final_inert = []
    
for sample in range(NUM_ATTEMPTS):
    print('\nCentroid attempt: ', sample)
    km = KMeans(n_clusters=NUM_CLUSTERS, init='random', max_iter=1, n_init=1)#, verbose=1) 
    km.fit(data_sample)
    inertia_start = km.inertia_
    intertia_end = 0
    cents = km.cluster_centers_
        
    for iter in range(NUM_ITER):
        km = KMeans(n_clusters=NUM_CLUSTERS, init=cents, max_iter=1, n_init=1)
        km.fit(data_sample)
        print('Iteration: ', iter)
        print('Inertia:', km.inertia_)
        print('Centroids:', km.cluster_centers_)
        inertia_end = km.inertia_
        cents = km.cluster_centers_

    final_cents.append(cents)
    final_inert.append(inertia_end)
    print('Difference between initial and final inertia: ', inertia_start-inertia_end)


Centroid attempt:  0
Iteration:  0
Inertia: 909.41638077
Centroids: [[ 0.55059892  0.10729486  0.36109086]
 [ 0.42275889  0.13662652  0.20143432]
 [ 0.57508514  0.33846074  0.2971163 ]
 [ 0.44444158  0.5318972   0.15649561]
 [ 0.55280816  0.41767155  0.10815682]
 [ 0.77293781  0.79176057  0.11370021]
 [ 0.16934451  0.23806167  0.18633921]
 [ 0.57031069  0.26379549  0.55919627]
 [ 0.66138394  0.69542807  0.31733501]
 [ 0.65663419  0.33947277  0.12062702]]
Iteration:  1
Inertia: 889.993859664
Centroids: [[ 0.56011985  0.10553658  0.3448233 ]
 [ 0.38584656  0.13921513  0.20812626]
 [ 0.56327522  0.34194838  0.30298824]
 [ 0.44345146  0.53279726  0.15341629]
 [ 0.55582455  0.41437762  0.11358421]
 [ 0.76896359  0.78348386  0.11817672]
 [ 0.16172169  0.24507193  0.18338347]
 [ 0.5716509   0.2700144   0.55639128]
 [ 0.65512919  0.6952581   0.33003951]
 [ 0.66617805  0.32363351  0.12027365]]
Iteration:  2
Inertia: 876.444163865
Centroids: [[ 0.56241546  0.10312694  0.33331749]
 [ 0.36449926 

Inertia : Sum of squares of distance from centroid to each point. <br>
When the centroids are almost converging the difference in the initial and final inertia is less. <br><br>
Step 3: we consider the centroids of the last iteration of the least difference in Inertia as the best centroids for our clustering algorithm

In [25]:
# Get best centroids to use for full clustering
best_cents = final_cents[final_inert.index(min(final_inert))]
best_cents

array([[ 0.23867275,  0.09290232,  0.29755624],
       [ 0.57613819,  0.16801781,  0.37714603],
       [ 0.68045596,  0.65548369,  0.4581411 ],
       [ 0.2750434 ,  0.39540308,  0.15424408],
       [ 0.51388455,  0.57042661,  0.16779373],
       [ 0.72630351,  0.77025118,  0.16108645],
       [ 0.61187835,  0.37879068,  0.14739282],
       [ 0.10968981,  0.23393846,  0.19278895],
       [ 0.43845147,  0.16227707,  0.23216506],
       [ 0.26282238,  0.1433795 ,  0.12333201]])

Running the K-Means algorithm with the best centroids found as the initial Centroids

In [26]:
km_full = KMeans(n_clusters=NUM_CLUSTERS, init=best_cents, max_iter=100, verbose=1, n_init=1)
km_full.fit(data)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 8615.69337846
start iteration
done sorting
end inner loop
Iteration 1, inertia 8447.30703369
start iteration
done sorting
end inner loop
Iteration 2, inertia 8289.22072713
start iteration
done sorting
end inner loop
Iteration 3, inertia 8159.99407089
start iteration
done sorting
end inner loop
Iteration 4, inertia 8069.93498105
start iteration
done sorting
end inner loop
Iteration 5, inertia 8004.91522631
start iteration
done sorting
end inner loop
Iteration 6, inertia 7949.82292208
start iteration
done sorting
end inner loop
Iteration 7, inertia 7885.72285927
start iteration
done sorting
end inner loop
Iteration 8, inertia 7835.87335243
start iteration
done sorting
end inner loop
Iteration 9, inertia 7774.56531578
start iteration
done sorting
end inner loop
Iteration 10, inertia 7699.89071194
start iteration
done sorting
end inner loop
Iteration 11, inertia 7610.61270858
start iteration
done sorti

KMeans(algorithm='auto', copy_x=True,
    init=array([[ 0.23867,  0.0929 ,  0.29756],
       [ 0.57614,  0.16802,  0.37715],
       [ 0.68046,  0.65548,  0.45814],
       [ 0.27504,  0.3954 ,  0.15424],
       [ 0.51388,  0.57043,  0.16779],
       [ 0.7263 ,  0.77025,  0.16109],
       [ 0.61188,  0.37879,  0.14739],
       [ 0.10969,  0.23394,  0.19279],
       [ 0.43845,  0.16228,  0.23217],
       [ 0.26282,  0.14338,  0.12333]]),
    max_iter=100, n_clusters=10, n_init=1, n_jobs=1,
    precompute_distances='auto', random_state=None, tol=0.0001, verbose=1)

Takes around 46 iterations for the centroids to Converge

Run K-Means algorithm on the dataset with random centroids as the initial centroids

In [27]:
km_naive = KMeans(n_clusters=NUM_CLUSTERS, init='random', max_iter=100, verbose=1, n_init=1)
km_naive.fit(data)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 10257.2370256
start iteration
done sorting
end inner loop
Iteration 1, inertia 8630.91808378
start iteration
done sorting
end inner loop
Iteration 2, inertia 8253.6269471
start iteration
done sorting
end inner loop
Iteration 3, inertia 8104.10087852
start iteration
done sorting
end inner loop
Iteration 4, inertia 8022.72296289
start iteration
done sorting
end inner loop
Iteration 5, inertia 7967.40238132
start iteration
done sorting
end inner loop
Iteration 6, inertia 7908.7181642
start iteration
done sorting
end inner loop
Iteration 7, inertia 7848.38017324
start iteration
done sorting
end inner loop
Iteration 8, inertia 7807.12915787
start iteration
done sorting
end inner loop
Iteration 9, inertia 7789.58410833
start iteration
done sorting
end inner loop
Iteration 10, inertia 7782.26327502
start iteration
done sorting
end inner loop
Iteration 11, inertia 7776.94669512
start iteration
done sorting

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)

Takes 85 iterations for the centroids to Converge. <br> <br>
Hence, the method of finding better centroids using above method has worked.